In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import spacy
from tensorflow import keras
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import re

In [2]:
df = pd.read_csv("./dataset/clean_rent_the_runway.csv")

In [3]:
def normalize_height(data): ##Convert into cms
    if pd.notnull(data):
        try:
            return (int(data[0])*30.48) + (int(data[4:-2])*2.54)
        except:
            return (int(data[0])*30.48)
        return None

SyntaxError: invalid syntax (3611035946.py, line 7)

In [ ]:
df["height"] = df["height"].apply(lambda x: normalize_height(x))

In [50]:
train[train['Label'] == 0]['Review Text'].values[0]

'I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c'

In [51]:
train[train['Label'] == 1]['Review Text'].values[0]

'Absolutely wonderful - silky and sexy and comfortable'

In [54]:
CONTRACTION_MAP = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [62]:
python -m spacy download en_core_web_sm

SyntaxError: invalid syntax (581980377.py, line 1)

In [61]:
nlp = spacy.load('en_core_web_sm')
sent = nlp.create_pipe('sentencizer')
nlp.add_pipe(sent, before='parser')
nlp.pipe_names

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
def clean_text(text) :
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = ' '.join([i for i in text.split() if '@' not in i])
    text = re.sub('http[s]?://\S+', '', text)
    for word in text.split():
        if word.lower() in CONTRACTION_MAP:
            text = text.replace(word[1:], CONTRACTION_MAP[word.lower()][1:])
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern, '', text)
    text = re.sub('\s+', ' ', text)
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.lemma_ != '-PRON-':
            tokens.append(token.lemma_.lower().strip())
        else:
            tokens.append(token.lower_)

    return ' '.join(tokens)

In [55]:
x_train = train['Review Text']
y_train = train['Label']
y_test = test['Review Text']

In [56]:
x_train.head(10)

0    Absolutely wonderful - silky and sexy and comf...
1    Love this dress!  it's sooo pretty.  i happene...
2    I had such high hopes for this dress and reall...
3    I love, love, love this jumpsuit. it's fun, fl...
4    This shirt is very flattering to all due to th...
5    I love tracy reese dresses, but this one is no...
6    I aded this in my basket at hte last mintue to...
7    I ordered this in carbon for store pick up, an...
8    I love this dress. i usually get an xs but it ...
9    I'm 5"5' and 125 lbs. i ordered the s petite t...
Name: Review Text, dtype: object

In [57]:
train_size = int(len(x_train) * 0.8)
index = np.random.permutation(x_train.index)
train_index = index[:train_size]
valid_index = index[train_size:]
train_data = x_train.iloc[train_index]
train_labels = y_train.iloc[train_index]
valid_data = x_train.iloc[valid_index]
valid_labels = y_train.iloc[valid_index]

In [58]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
vocab_size = len(word_index)

AttributeError: 'float' object has no attribute 'lower'

In [44]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
base_model = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [45]:
model = tf.keras.Sequential([
    base_model,
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1),
])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_15 (Dense)            (None, 128)               6528      
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dense_17 (Dense)            (None, 32)                2080      
                                                                 
 dense_18 (Dense)            (None, 16)                528       
                                                                 
 dense_19 (Dense)            (None, 1)                 17        
                                                                 
Total params: 48,208,009
Trainable params: 48,208,009


In [46]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [47]:
history = model.fit(x_train,y_train,
                    epochs=10,
                    validation_split = 0.3,
                    verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [8]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 5s - loss: 0.7240 - accuracy: 0.8427 - 5s/epoch - 107ms/step
loss: 0.724
accuracy: 0.843


In [22]:
input =["bad!!!"]
model.predict(input)

1/1 [==============================] - 0s 136ms/step


array([[-0.28167397]], dtype=float32)

In [19]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>